# TP 3 : Implémentation de l'algorithme du pivot de Gauss

La méthode du pivot de Gauss est une méthode générale de résolution d'un système linéaire de la forme $AX=B$ où $A$ est la matrice des coefficients et $B$ la colonne des seconds membres. 

Nous nous limiterons dans ce TP à la résolution des systèmes linéaires **carrés** (autant d'équations que d'inconnues) et admettant **une unique solution** : nous avons appris que cela signifie que la matrice $A$ est inversible. 

Ces systèmes sont appelés des **systèmes de Cramer**.


Voici un exemple d'un tel système : 
 
$$\left\{ \begin{aligned}
x_1-x_2+2x_3+x_4& =1 \\
2x_1-3x_2+5x_3+2x_4& =3 \\
3x_1+2x_2+x_3+2x_4& =0 \\
x_1+x_2-x_3-3x_4& =0
\end{aligned}
\right.  , \quad \text{ c'est-à-dire } \quad \underbrace{\begin{pmatrix} 1&-1&2&1\\2&-3&5&2\\3&2&1&2\\1&1&-1&-3  \end{pmatrix}}_{A} \times  \underbrace{\begin{pmatrix} x_1\\x_2\\x_3\\x_4 \end{pmatrix}}_{X} = \underbrace{\begin{pmatrix} 1\\3\\0\\0 \end{pmatrix}}_{B} \quad \text{ qui admet l'unique solution } (-5,6,7,-2)$$


Cette méthode repose sur l'application *d'opérations élémentaires sur les lignes* des matrices $A$ et $B$. 

Nous avons vu en effet qu'on ne modifie pas l'ensemble des solutions en appliquant les mêmes opérations élémentaires sur les lignes de $A$ et de $B$.

Dans ce TP, comme au TP précédent, une matrice sera stockée comme un objet de type *liste* contenant les listes de ces lignes. 

----



In [25]:
def display(A):
    for ele in A:
        print(ele)

#### Exercice 1 : Opérations élémentaires sur les lignes d'une matrice

Pour implémenter l'algorithme du pivot de Gauss, nous avons besoin de programmer les trois fonctions correspond aux trois opérations élémentaires sur les lignes d'une matrice : 

1. la fonction `permLigne()` qui permute deux lignes d'une matrice ($L_i \leftrightarrow L_j$)
2. la fonction `multLigne()` qui multiplie tous les éléments d'une ligne par un nombre ($L_i \leftarrow k L_i$)
3. la fonction `addLigne()` qui ajoute à une ligne un multiple d'une autre ligne ($L_j \leftarrow L_j + k  L_i$) 

In [1]:
# En entrée : A une liste de listes et i,j deux indices
# Ne renvoie rien mais modifie la liste A en permutant ses listes d'indice i et j 

def permLigne(A,i,j):
    A[i], A[j] = A[j], A[i]
    # à compléter
    

In [2]:
# test de la fonction permLigne()
A = [[1,1,1], [2,2,2], [3, 3, 3]]
permLigne(A, 1, 2)
print(A)

[[1, 1, 1], [3, 3, 3], [2, 2, 2]]


In [4]:
# En entrée : A une liste de listes, k un nombre et i un indice
# Ne renvoie rien mais modifie la liste A en multipliant tous les éléments de sa liste i par k : Li <- k * Li

def multLigne(A,k,i):
    for j in range(len(A[i])):
        A[i][j]*=k

In [5]:
# test de la fonction multLigne()
multLigne(A, 4, 0)
print(A)

[[4, 4, 4], [3, 3, 3], [2, 2, 2]]


In [7]:
# En entrée : A une liste de listes, k un nombre et i,j deux indices
# Ne renvoie rien mais modifie la liste A en ajoutant à sa liste i k-fois sa liste j  : Li <- Li + k * Lj 

def addLigne(A,k,i,j):
    for l in range(len(A[i])):
        A[i][l] += k*A[j][l]

In [8]:
# test de fonction addLigne()
addLigne(A, 3, 0, 2)
print(A)

[[10, 10, 10], [3, 3, 3], [2, 2, 2]]


----

La méthode du pivot de Gauss appliqué à un système de Cramer $AX=B$ comporte trois étapes :
1. une premiere étape dite *de descente* qui consiste à transformer la matrice inversible $A$ en une matrice $A'$ triangulaire supérieure (avec éléments diagonaux non nuls) tout en effectuant les mêmes opérations sur $B$ : 

<img src="Etape1_descente.png" width="300">

2. une deuxième étape dite *de remontée* qui consiste à transformer la matrice inversible $A'$ en une matrice $A''$ diagonale (avec éléments diagonaux non nuls) tout en effectuant les mêmes opérations sur B : 

<img src="Etape2_remontee.png" width="300">


3. Une troisième étape de résolution du système linéaire, devenu trivial car diagonal. En transformant la matrice diagonale $A''$ en la matrice Identité (on ramène les pivots à 1), la solution est immédiate. 


Vous trouverez dans le document `exempleGauss.pdf` l'appplication détaillée de cette méthode sur le système cité en exempple : 

$$\left\{ \begin{aligned}
x_1-x_2+2x_3+x_4& =1 \\
2x_1-3x_2+5x_3+2x_4& =3 \\
3x_1+2x_2+x_3+2x_4& =0 \\
x_1+x_2-x_3-3x_4& =0
\end{aligned}
\right.  , \quad \text{ c'est-à-dire } \quad \underbrace{\begin{pmatrix} 1&-1&2&1\\2&-3&5&2\\3&2&1&2\\1&1&-1&-3  \end{pmatrix}}_{A} \times  \underbrace{\begin{pmatrix} x_1\\x_2\\x_3\\x_4 \end{pmatrix}}_{X} = \underbrace{\begin{pmatrix} 1\\3\\0\\0 \end{pmatrix}}_{B} \quad \text{ qui admet l'unique solution } (-5,6,7,-2)$$
----


#### Exercice 2  : Etape de descente

Cette étape consiste à transformer progressivement la matrice $A$ par opérations élémentaires sur les lignes pour obtenir une matrice triangulaire supérieure : 
$$ \text{ à la $j$-ie itération :  \quad } 
\begin{pmatrix} 
a_{11} & a_{12} & \cdots & \cdots & \cdots & a_{1n} \\
0 & a_{22} & \cdots & \cdots & \cdots & a_{2n} \\
\vdots & \ddots & \ddots &  &   & \vdots \\
0 & \cdots & 0 & a_{jj} & \cdots & a_{jn} \\
\vdots &   & \vdots & \vdots &  & \vdots \\
0 & \vdots & 0 & a_{nj} & \cdots & a_{nn} \\
 \end{pmatrix}$$
 - on cherche un pivot non nul $a_{kj}$ sur la colonne $j$ parmi les éléments $a_{j,j} \quad a_{j+1,j} \quad \cdots \quad  a_{n,j}$ puis on permute les lignes $L_k$ et $L_j$
 - à l'issue de cette première étape, on est assuré que $a_{jj} \neq 0$ : on l'utilise comme pivot pour remplacer tous les termes $a_{ij} $ pour $i>j$ par des zéros à l'aide de l'opération élémentaire : 
 $$ \forall i > j, \quad L_i \leftarrow L_i - \dfrac{a_{ij}}{a_{jj}}L_j$$ 


 Cela correspond à l'algorithme : 
```
A la matrice des coefficients du système
B la colonne des seconds membres
n le nombre de ligne ou colonne de la matrice A

Pour j variant de 1 à ????? : 
     Trouver k entre j et n tel que a_kj non nul (le pivot)
     Permuter dans A et dans B les lignes Lk et Lj
     pour i variant de ????? à n : 
       Remplacer la ligne Li par Li - (a_ij/a_jj)* Lj
```

1. Créer une fonction `recherchePivot(A,j)` qui retourne l'indice de ligne du premier élément non nul sur la colonne $j$ parmi : 
$$a_{j,j} \quad a_{j+1,j} \quad \cdots \quad  a_{n,j}$$

In [62]:
def recherchePivot(A,j):
    # à compléter
    long = len(A)
    indice = j
    while A[indice][j]==0 and indice < long:
        indice += 1
    print(indice)
    return indice

In [14]:
# test de recherchePivot sur ces 4 cas
A=[[0,1,1,2],[0,2,4,7],[2,0,0,8],[3,4,6,9]]
print(recherchePivot(A,0)) # retourne 2
print(recherchePivot(A,1)) # retourne 1
print(recherchePivot(A,2)) # retourne 3
print(recherchePivot(A,3)) # retourne 3


2
1
3
3


2. Créer une fonction `descente(A,B)` qui réalise l'étape de descente de la méthode de Gauss sur le système $AX=B$

In [23]:
# ne retourne rien mais modifie les listes A et B
def descente(A,B):
    for j in range(len(A)):
        indice = recherchePivot(A, j)
        permLigne(A, j, indice)
        permLigne(B, j, indice)
        for i in range(j+1, len(A)):
            k = -(A[i][j]/A[j][j])
            addLigne(A, k, i, j)
            addLigne(B, k, i, j)
    # à compléter      

In [26]:
# test (voir le résultat attendu dans le fichier exempleGauss.pdf)
A=[[1,-1,2,1],[2,-3,5,2],[3,2,1,2],[1,1,-1,-3]]
B=[[1],[3],[0],[0]]
descente(A,B)
display(A)
display(B)

[1, -1, 2, 1]
[0.0, -1.0, 1.0, 0.0]
[0.0, 0.0, -1.0, -4.0]
[0.0, 0.0, 0.0, -1.0]
[1]
[1.0]
[1.0]
[2.0]


----
#### Exercice 3  : Etape de remontée 

L'étape de remontée intervient lorsque la matrice $A$ est triangulaire supérieur, les coefficients de la diagonale étant non nuls. 

On transforme progressivement la matrice A de façon itérative, en partant de la dernière ligne et en faisant apparaitre les zéros au dessus de chaque pivot :

$$ \text{ à la $n-j$-ie itération :  \quad } 
\begin{pmatrix} 
a_{11} & \cdots & a_{1j} & 0 & \cdots & 0 \\
0      & \ddots & \vdots & \vdots &  & \vdots \\
\vdots & \ddots & a_{jj} & 0 &   & \vdots \\
0 & \cdots & 0 & a_{j+1,j+1} & \ddots & \vdots \\
\vdots &   & \vdots & \ddots & \ddots & 0 \\
0 & \cdots & 0 & \cdots & 0 & a_{nn} \\
 \end{pmatrix}$$

 à l'issue de cette première étape, on est assuré que $a_{jj} \neq 0$ : on utilise le pivot $a_{jj}$ pour remplacer tous les termes $a_{ij} $ pour $i<j$ ($a_{1,j}, a_{2,j}, \cdots, a_{j-1,j}$ ) par des zéros à l'aide de l'opération élémentaire : 
 $$ \forall i < j, \quad L_i \leftarrow L_i - \dfrac{a_{ij}}{a_{jj}}L_j$$ 




La phase de diagonalisation correspond à l'algorithme suivant : 

```
Pour j variant de n à ????? : 
     pour i variant de 1 à ???? : 
            Remplacer la ligne Li par Li - (a_ij/a_jj)* Lj
```


Créer une fonction `remontee(A,B)` qui réalise l'étape de remontée de la méthode de Gauss sur les matrices A et B 

In [41]:
# ne retourne rien mais modifie les listes A et B
def remontee(A,B):
    for j in range(len(A)-1, 0, -1):
        for i in range(0, j):
            k = -(A[i][j]/A[j][j])
            addLigne(A, k, i, j)
            addLigne(B, k, i, j)
    # à compléter

In [42]:
# test (voir le résultat attendu dans le fichier exempleGauss.pdf)
A=[[1,-1,2,1],[2,-3,5,2],[3,2,1,2],[1,1,-1,-3]]
B=[[1],[3],[0],[0]]
descente(A,B)
remontee(A,B)
display(A)
display(B)

[1.0, 0.0, 0.0, 0.0]
[0.0, -1.0, 0.0, 0.0]
[0.0, 0.0, -1.0, 0.0]
[0.0, 0.0, 0.0, -1.0]
[-5.0]
[-6.0]
[-7.0]
[2.0]


-----
#### Exercice 4  : Étape de résolution du système 

La troisème et dernière étape de la méthode consiste à transformer le système diagonal $AX=B$ obtenu après les deux étapes précédentes en $I_n X = B'$ : on ramène chaque élément diagonal de A (les pivots) à 1. 

1. Écrire une fonction `solve_diagonal(A,B)`qui prend en arguments une matrice diagonale inversible $A$ et une matrice-colonne $B$ et qui retourne l'unique nuplet X solution de l'équation $AX=B$. 

In [48]:
def solve_diagonale(A,B):
    for i in range(len(A)):
        B[i][0]*=A[i][i]
    return B

In [47]:
# test 
A=[[1,-1,2,1],[2,-3,5,2],[3,2,1,2],[1,1,-1,-3]]
B=[[1],[3],[0],[0]]
descente(A,B)
remontee(A,B)
solve_diagonale(A,B)

[[-5.0], [6.0], [7.0], [-2.0]]

2. En déduire une fonction `gauss(A,B)` qui reprend les trois étapes précédentes pour retourner l'unique solution d'un système de Cramer $AX+B$. 

In [49]:
def gauss(A,B):
    # On travaille sur des copies des matrices A et B 
    # pour éviter de modifier physiquement ces dernières 
    U=A.copy()
    V=B.copy()
    # à compléter
    descente(U, V)
    remontee(U, V)
    solve_diagonale(U, V)
    return V

    


In [50]:
# test (voir le résultat attendu dans le fichier exempleGauss.pdf)
A=[[1,-1,2,1],[2,-3,5,2],[3,2,1,2],[1,1,-1,-3]]
B=[[1],[3],[0],[0]]
soluce = gauss(A,B)
print(soluce)

[[-5.0], [6.0], [7.0], [-2.0]]


----
#### Exercice 5  : Méthode du pivot partiel 

Une alternative à la méthode du pivot de Gauss consiste à choisir comme pivot  sur la colonne $j$ parmi $a_{j,j} , a_{j+1,j} ,\cdots , a_{n,j}$ le plus grand élément (en valeur absolue)

Ecrire une nouvelle fonction `recherchePivotPartiel()` et une nouvelle fonction `gaussPartiel()` pour mettre en place cette méthode alternative. 


In [95]:
def valAbs(i):
    return -i if (i < 0) else i

def recherchePivotPartiel(A,j):
    #  à compléter
    long = len(A)
    indice = j
    for i in range(j+1, long):
        if valAbs(A[i][j]) > valAbs(A[indice][j]):
            indice = i
    return indice

def descentePartielle(A, B):
    for j in range(len(A)):
        indice = recherchePivotPartiel(A, j)
        permLigne(A, j, indice)
        permLigne(B, j, indice)
        for i in range(j+1, len(A)):
            k = -(A[i][j]/A[j][j])
            addLigne(A, k, i, j)
            addLigne(B, k, i, j)

def gaussPartiel(A,B):
    # à compléter
    U = A.copy()
    V = B.copy()
    descentePartielle(U, V)
    remontee(U, V)
    solve_diagonale(U, V)
    display(U)
    return V


In [96]:
A=[[1,-1,2,1],[2,-3,5,2],[3,2,1,2],[1,1,-1,-3]]
B=[[1],[3],[0],[0]]
soluce = gaussPartiel(A,B)

print(soluce)

[3.0, 0.0, 0.0, 0.0]
[0.0, -4.333333333333377, 0.0, 0.0]
[0.0, -1.0436096431476689e-14, -0.9999999999999999, 0.0]
[0.0, -2.220446049250313e-16, 0.0, 0.07692307692307532]
[[-45.00000000000056], [112.66666666667034], [7.000000000000139], [-0.011834319526626955]]
